In [ ]:
!/opt/bin/nvidia-smi

Tue Oct 26 21:32:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    37W / 300W |  12223MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#For A100-SXM4-40GB
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch
print(torch.__version__)

1.9.0+cu111


In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir('/content/gdrive/My Drive/Interactive/Code/Data')
!unzip annotated_test_images.zip

In [ ]:
!unzip FSC147_384_V2.zip

#Main

In [ ]:
os.chdir('/content/gdrive/My Drive/Interactive Counting/')
import os
import cv2
import copy
import json
import torch
import random
import datetime
import numpy as np
from utils import matlab_style_gauss2D
from region_sum_less_4 import VIS
from tqdm import tqdm
import torch.optim as optim
import torch.nn.functional as F
from dataset import FscBgDataset
from Kernel_Resdiual_Learning import resdiual_learning
from utils import extract_features, TransformTrain, MincountLoss, PerturbationLoss
from model import Resnet50FPN, FixedCountRegressor, AdaptedCountRegressor, CountRegressor, weights_normal_init

In [ ]:
Root_dir = '/content/gdrive/My Drive/Interactive/Code/Data/'
Save_dir = '/content/gdrive/My Drive/Interactive Counting/Saves/'
Model_dir = os.path.join(Root_dir, 'FamNet_Pretrained.pth')
train_dataset = FscBgDataset(Root_dir, 'train', False)
val_dataset = FscBgDataset(Root_dir, 'val', False)
test_dataset = FscBgDataset(Root_dir, 'test', False)
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = 'cpu'
resnet50_conv = Resnet50FPN()
resnet50_conv.to(device)
regressor = CountRegressor(6, pool='mean')
regressor.load_state_dict(torch.load(Model_dir))
regressor.to(device)
resnet50_conv.eval()
regressor.eval()

In [ ]:
def interactive_loss(density, gt_density, mask, count_limit = 4):
    density = density * mask
    gt_density = gt_density * mask
    if gt_density.sum() >= count_limit:
        loss = max(0, count_limit - density.sum())
    else:
        loss = max(0, gt_density.sum() - density.sum()) + max(0, density.sum() - gt_density.sum())
    return loss

def sample_pixel(label):
  height, width = label.shape
  y = random.randint(0, height - 1)
  x = random.randint(0, width - 1)
  return y,x

def random_region_sample(label):
  max_label = np.max(label) + 1
  random_label = np.random.randint(0, int(max_label))
  return random_label

def error_region_sample(label, density, gt_density):
  max_label = np.max(label) + 1
  max_error = 0
  final_sample_label = None
  #For all region, calculate the loss
  for sample_label in range(max_label):
    #The Mask
    inter_mask = np.zeros((label.shape[0], label.shape[1]), dtype=np.uint8)
    inter_mask[label == sample_label] = 1
    inter_mask = torch.from_numpy(inter_mask).to(device)
    #Error
    inter_error = interactive_loss(density, gt_density, inter_mask)
    #Max error
    if max_error < inter_error:
      max_error = inter_error
      final_sample_label = sample_label
  return final_sample_label

#Random

In [ ]:
def adapted_test():
    inter_result = [[], [], [], [], []]
    show_detail = False
    print("Adapted Test on FSC147 train set data")
    MAPS = ['map3', 'map4']
    Scales = [0.9, 1.1]
    idx_list = list(range(len(val_dataset)))
    SAE = 0
    SSE = 0
    cnt = 0
    starttime = datetime.datetime.now()
    criterion = torch.nn.MSELoss().cuda()
    
    for idx in idx_list:
        test_sample = val_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = test_sample['im_id'], test_sample['image'], test_sample['boxes'], test_sample['dots'], test_sample['bg_mask_img'], test_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, gt_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)

        with torch.no_grad():
            features = extract_features(resnet50_conv, image.unsqueeze(0), boxes.unsqueeze(0), MAPS, Scales)
        adapted_regressor = copy.deepcopy(regressor)

        '''
        Do the Test-Adaptation
        '''

        if Test_Adaptation:
          adapted_regressor.train()
          optimizer = optim.Adam(adapted_regressor.parameters(), lr=ADLR)
          features.required_grad = True
          for step in range(0, GS):
              optimizer.zero_grad()
              output = adapted_regressor(features)
              lCount = weight_mincount * MincountLoss(output, boxes)
              lPerturbation = weight_perturbation * PerturbationLoss(output, boxes, sigma=8)
              loss = lCount + lPerturbation
              if torch.is_tensor(loss):
                  loss.backward()
                  optimizer.step()

        '''
        Interaction
        '''
        inter_mask_list = []
        for int_time in range(Inter_time):
            #Inference
            features.required_grad = False
            output = adapted_regressor(features)

            #VIS
            density = output.squeeze().detach().cpu().numpy()
            visual = VIS(density)
            visual.solve()
            label = visual.Llabel

            #Sample Region Randomly
            sample_label = random_region_sample(label)

            #Get Mask
            inter_mask = np.zeros((label.shape[0], label.shape[1]), dtype=np.uint8)
            inter_mask[label == sample_label] = 1
            inter_mask = torch.from_numpy(inter_mask).to(device)
            inter_mask_list.append(inter_mask)
            #Reshape The GT
            if int_time < 1:
                gt_density = F.interpolate(gt_density, size=(inter_mask.shape[0], inter_mask.shape[1]),
                                           mode='bilinear').squeeze()

            #Updating with Interaction
            optimizer_inter = optim.Adam(adapted_regressor.parameters(), lr=INLR)
            features.required_grad = True
            for step in range(0, INGS):
                #Inter Opti
                optimizer_inter.zero_grad()
                output = adapted_regressor(features).squeeze()
                #Scheme 3
                inter_loss = interactive_loss(output, gt_density, inter_mask)
                if torch.is_tensor(inter_loss):
                    inter_loss.backward()
                    optimizer_inter.step()

            '''
            After Interaction
            '''

            features.required_grad = False
            output = adapted_regressor(features).squeeze()
            pred_cnt = output.sum().item()
            gt_cnt = dots.shape[0]
            cnt = cnt + 1
            err = gt_cnt - pred_cnt
            inter_result[int_time].append(err)
        assert len(inter_mask_list) == 5
    return inter_result

###############################################Adaptation##############################################
ADLR = 1e-7
GS = 100
weight_mincount = 1e-9
weight_perturbation = 1e-4

###############################################Interaction#############################################
Inter_time = 5
INGS = 15
INLR = 1e-6
Test_Adaptation = True
show_detail = False



Inter_result = adapted_test()
for inter_time in range(Inter_time):
    image_errs = Inter_result[inter_time]
    image_errs = np.array(image_errs)
    mse = np.sqrt(np.mean(np.square(image_errs)))
    mae = np.mean(np.abs(image_errs))
    print('mae {}, mse {}\n'.format(mae, mse))


Adapted Test on FSC147 train set data


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/content/gdrive/My Drive/Interactive Counting/utils.py:297: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  Loss += F.mse_loss(X,ones)


mae 21.470863535606288, mse 65.85469470736172

mae 19.836008624331576, mse 62.996610828623474

mae 18.745018058162454, mse 61.45879279794624

mae 18.08704783845096, mse 61.00249762303504

mae 17.701802071062254, mse 61.3224876499629



#Error

In [ ]:
def adapted_test():
    inter_result = [[], [], [], [], []]
    show_detail = False
    print("Adapted Test on FSC147 train set data")
    MAPS = ['map3', 'map4']
    Scales = [0.9, 1.1]
    idx_list = list(range(len(val_dataset)))
    SAE = 0
    SSE = 0
    cnt = 0
    starttime = datetime.datetime.now()
    criterion = torch.nn.MSELoss().cuda()
    
    for idx in idx_list:
        test_sample = val_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = test_sample['im_id'], test_sample['image'], test_sample['boxes'], test_sample['dots'], test_sample['bg_mask_img'], test_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, gt_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)

        with torch.no_grad():
            features = extract_features(resnet50_conv, image.unsqueeze(0), boxes.unsqueeze(0), MAPS, Scales)
        adapted_regressor = copy.deepcopy(regressor)

        '''
        Do the Test-Adaptation
        '''

        if Test_Adaptation:
          adapted_regressor.train()
          optimizer = optim.Adam(adapted_regressor.parameters(), lr=ADLR)
          features.required_grad = True
          for step in range(0, GS):
              optimizer.zero_grad()
              output = adapted_regressor(features)
              lCount = weight_mincount * MincountLoss(output, boxes)
              lPerturbation = weight_perturbation * PerturbationLoss(output, boxes, sigma=8)
              loss = lCount + lPerturbation
              if torch.is_tensor(loss):
                  loss.backward()
                  optimizer.step()

        '''
        Interaction
        '''
        inter_mask_list = []
        for int_time in range(Inter_time):
            #Inference
            features.required_grad = False
            output = adapted_regressor(features)

            #VIS
            density = output.squeeze().detach().cpu().numpy()
            visual = VIS(density)
            visual.solve()
            label = visual.Llabel
            
            #Reshape The GT
            if int_time < 1:
                gt_density = F.interpolate(gt_density, size=(label.shape[0], label.shape[1]),
                                           mode='bilinear').squeeze()

            #Sample Region Randomly
            #sample_label = random_region_sample(label)
            sample_label = error_region_sample(label, output.squeeze(), gt_density)

            #Get Mask
            inter_mask = np.zeros((label.shape[0], label.shape[1]), dtype=np.uint8)
            inter_mask[label == sample_label] = 1
            inter_mask = torch.from_numpy(inter_mask).to(device)
            inter_mask_list.append(inter_mask)
            

            #Updating with Interaction
            optimizer_inter = optim.Adam(adapted_regressor.parameters(), lr=INLR)
            features.required_grad = True
            for step in range(0, INGS):
                #Inter Opti
                optimizer_inter.zero_grad()
                output = adapted_regressor(features).squeeze()
                #Scheme 3
                inter_loss = interactive_loss(output, gt_density, inter_mask)
                if torch.is_tensor(inter_loss):
                    inter_loss.backward()
                    optimizer_inter.step()

            '''
            After Interaction
            '''

            features.required_grad = False
            output = adapted_regressor(features).squeeze()
            pred_cnt = output.sum().item()
            gt_cnt = dots.shape[0]
            cnt = cnt + 1
            err = gt_cnt - pred_cnt
            inter_result[int_time].append(err)
        assert len(inter_mask_list) == 5
    return inter_result

###############################################Adaptation##############################################
ADLR = 1e-7
GS = 100
weight_mincount = 1e-9
weight_perturbation = 1e-4

###############################################Interaction#############################################
Inter_time = 5
INGS = 15
INLR = 1e-6
Test_Adaptation = True
show_detail = False



Inter_result = adapted_test()
for inter_time in range(Inter_time):
    image_errs = Inter_result[inter_time]
    image_errs = np.array(image_errs)
    mse = np.sqrt(np.mean(np.square(image_errs)))
    mae = np.mean(np.abs(image_errs))
    print('mae {}, mse {}\n'.format(mae, mse))


Adapted Test on FSC147 train set data


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/content/gdrive/My Drive/Interactive Counting/utils.py:297: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  Loss += F.mse_loss(X,ones)


mae 19.333271818806185, mse 64.41221646373836

mae 17.61154621199909, mse 61.92246654294099

mae 15.700249072367074, mse 57.65492074504789

mae 15.931036880932181, mse 56.422314540826534

mae 14.979593455698776, mse 55.661087816311216

